# W03a Hotel Booking - Python and Sqlite3

An hotel would like to implement a web application to manage their room booking online.

The following information of each `RoomType` is stored:
- `RoomTypeID` – unique auto increment int to identify the type of room
- `RoomType` – currently room type can only be either `Deluxe Twin`, `Deluxe King` or `Suite`
- `Price` – original price for the room
- `Promotion` – promotion percentage for the room, defaulted as `1.0`, can range from `0.1` to `1.0`.
- `RoomImage` – optional field, url link to image file for the room type

The following information of each `Room` is stored:
- `RoomID` – unique auto increment int to identify each room in the hotel.
- `Level` – level of the room, e.g `07`
- `Unit` – unit of the room, e.g `18`
- `View` – view can only be either sea view, pool view or garden view
- `RoomTypeID` – room type id

The following information of each `Customer` is stored:
- `CustomerID` – unique string to identify the customer
- `Name` – name of customer
- `PassportNo` – passport number
- `DoB` – date of birth of customer
- `Country` – country of customer
- `Address` – address of customer

The following information of each `BookingRecord` is stored:
- `BookingNo` – unique autoincrement integer to identify the booking
- `RoomID` – room id
- `CustomerID` – customer ID
- `StartDate` – Date to start
- `NoOfDays` – No. of days the customer will stay in the hotel for this booking
- `PaymentStatus` – can be either unpaid or paid

The information is to be stored in above mentioned four tables.

In [20]:
# Run to link to db and unlock db when needed
import sqlite3
import csv

# !NOTE: if the db file does not exist, it will be created
db_file = "hotel.db"
conn = sqlite3.connect(db_file)
conn.close()

### Q1
Use DB Browser for SQLite to create the database schema for the tables mentioned above.

Save the database as `hotel.db`.

Copy and save the SQL script used to create the tables in the cell below.


In [21]:
# Q1 Answer:

query1 = """
CREATE TABLE IF NOT EXISTS "RoomType" (
	"RoomTypeID"	INTEGER,
	"RoomType"	TEXT NOT NULL CHECK("RoomType" = "Deluxe Twin" OR "RoomType" = "Deluxe King" OR "RoomType" = "Suite"),
	"Price"	REAL NOT NULL,
	"Promotion"	REAL NOT NULL DEFAULT 1,
	"RoomImage"	TEXT,
	PRIMARY KEY("RoomTypeID" AUTOINCREMENT)
);
"""
query2 = """
CREATE TABLE IF NOT EXISTS "Room" (
	"RoomID"	INTEGER,
	"Level"	TEXT NOT NULL,
	"Unit"	TEXT NOT NULL,
	"View"	TEXT NOT NULL CHECK("View" = "Sea View" OR "View" = "Pool View" OR "View" = "Garden View"),
	"RoomTypeID"	INTEGER NOT NULL,
	PRIMARY KEY("RoomID" AUTOINCREMENT)
);
"""
query3 = """
CREATE TABLE IF NOT EXISTS "Customer" (
	"CustomerID"	TEXT,
	"Name"	TEXT NOT NULL,
	"PassportNo"	TEXT NOT NULL,
	"DoB"	TEXT NOT NULL,
	"Country"	TEXT NOT NULL,
	"Address"	TEXT NOT NULL,
	PRIMARY KEY("CustomerID")
);
"""

query4 = """
CREATE TABLE IF NOT EXISTS "BookingRecord" (
	"BookingNo"	INTEGER,
	"RoomID"	INTEGER NOT NULL,
	"CustomerID"	TEXT NOT NULL,
	"StartDate"	TEXT NOT NULL,
	"NoOfDays"	INTEGER NOT NULL,
	"PaymentStatus"	TEXT NOT NULL DEFAULT Unpaid CHECK("PaymentStatus" = "Paid" OR "PaymentStatus" = "Unpaid"),
	FOREIGN KEY("RoomID") REFERENCES "Room"("RoomID"),
	FOREIGN KEY("CustomerID") REFERENCES "Customer"("CustomerID"),
	PRIMARY KEY("BookingNo" AUTOINCREMENT)
);
"""

queries = [query1, query2, query3, query4]

#### Q2

Write python code to create the database and create the tables mentioned above in the database.

In [22]:
# Q2 Answer:
conn = sqlite3.connect("hotel.db")
for query in queries:
    conn.execute(query)
conn.commit()
conn.close()

#### Q3
Read data from the csv files in the `data_files` folder, and insert the data into the respective tables.

In [23]:
# Q3 Answer:

files = {"BookingRecord": "bookingrecords.csv", "Customer": "customers.csv", "Room": "rooms.csv", "RoomType": "roomtypes.csv"}

for file in files:
    with open(f"data_files/{files[file]}", "r") as f:
        csv_reader = csv.reader(f)
        header = next(csv_reader)
        # print(header)
        data = []
        for row in csv_reader:
            conn = sqlite3.connect("hotel.db")
            ph = ",".join(["?" for _ in header])
            # print(ph)
            query = f"""
            INSERT INTO {file}
            {tuple(header)}
            VALUES
            ({ph})
            """
            # print(query)
            print(tuple(header))
            print(tuple(row))
            try:
                conn.execute(query, tuple(row))
            except sqlite3.Error as e:
                if 'UNIQUE constraint failed' in str(e):
                    print(f"Row already inserted. Skipping row: {row}")
                    continue
                else:
                    print(f"Error inserting row: {row}")
                    print(query)
                    print(e)
                    break
            conn.commit()
            conn.close()

('BookingNo', 'RoomID', 'CustomerID', 'StartDate', 'NoOfDays', 'PaymentStatus')
('1', '5', 'customer001', '20190901', '3', 'Paid')
('BookingNo', 'RoomID', 'CustomerID', 'StartDate', 'NoOfDays', 'PaymentStatus')
('2', '2', 'customer004', '20190901', '3', 'Paid')
('BookingNo', 'RoomID', 'CustomerID', 'StartDate', 'NoOfDays', 'PaymentStatus')
('3', '4', 'customer003', '20190901', '2', 'Paid')
('BookingNo', 'RoomID', 'CustomerID', 'StartDate', 'NoOfDays', 'PaymentStatus')
('4', '2', 'customer001', '20190904', '2', 'Unpaid')
('BookingNo', 'RoomID', 'CustomerID', 'StartDate', 'NoOfDays', 'PaymentStatus')
('5', '3', 'customer007', '20190908', '3', 'Paid')
('BookingNo', 'RoomID', 'CustomerID', 'StartDate', 'NoOfDays', 'PaymentStatus')
('6', '16', 'customer006', '20190910', '4', 'Paid')
('BookingNo', 'RoomID', 'CustomerID', 'StartDate', 'NoOfDays', 'PaymentStatus')
('7', '19', 'customer008', '20190914', '2', 'Paid')
('BookingNo', 'RoomID', 'CustomerID', 'StartDate', 'NoOfDays', 'PaymentStatus')

#### Q4

Write a python function to generate the unpaid outstanding bills for customer, based on `customer_name`, e.g. `Derren Brett`.

The query should return details including the following fields and sorted according to `StartDate` in ascending order:
- `Level, Unit, RoomType, Price, Promotion, StartDate, NoOfDays`


In [25]:
# Q4 Answer:
def gen_outstanding_bills(customer_name):
    conn = sqlite3.connect("hotel.db")
    query = """
    SELECT Level, Unit, RoomType, Price, Promotion, StartDate, NoOfDays
    FROM Room
    INNER JOIN RoomType
    ON Room.RoomTypeID == RoomType.RoomTypeID
    INNER JOIN BookingRecord
    ON Room.RoomID == BookingRecord.RoomID
    INNER JOIN Customer
    ON BookingRecord.CustomerID == Customer.CustomerID
    WHERE PaymentStatus == 'Unpaid'
    AND Name == ?
    ORDER BY BookingRecord.StartDate ASC
    """
    cursor = conn.execute(query, (customer_name,))
    results = cursor.fetchall()
    conn.close()
    return results

print(gen_outstanding_bills("Derren Brett"))

[('1', '2', 'Deluxe Twin', 350.0, 0.95, '20190904', 2), ('3', '6', 'Deluxe Twin', 350.0, 0.95, '20190917', 1)]
